<a href="https://colab.research.google.com/github/cool60334/Team_Project_Model/blob/main/p01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [3]:
# 讀取csv檔
f = pd.read_csv("", encoding = "utf-8")  # 輸入檔案名稱
pd.set_option('display.max_columns', 100)

In [ ]:
f

In [5]:
f = f[f['備註'].isnull()] # 刪除有備註的欄位

In [ ]:
f

# 移除不必要欄位

In [7]:
# 移除不必要欄位
f = f.drop(["非都市土地使用分區", "非都市土地使用編定", "交易筆棟數", "建物現況格局-隔間", "備註", "移轉編號", "編號"], axis = 1)

In [9]:
# 移除交易標的 土地
f = f[f["交易標的"] != "土地"]

# 重設index
f = f.reset_index(drop=True)

In [11]:
# 移除交易標的 車位
f = f[f["交易標的"] != "車位"]

# 重設index
f = f.reset_index(drop=True)

# 處理都市土地使用分區

In [29]:
# 移除其它都市土地使用分區

s = f["都市土地使用分區"]
for i in range(len(f)):
  if "其他" in str(s[i]):
    f = f.drop(i)

# 重設index
f = f.reset_index(drop=True)

In [ ]:
# 填補空值

f["都市土地使用分區"] = f["都市土地使用分區"].fillna("住")

# 處理樓層

In [31]:
column_dict = {"--":None, "":None, None:0, '陽台':1, '平台':1, '屋頂突出物':1, '夾層':1, '電梯樓梯間':1, '騎樓':1, '陽臺':1, '地下三層':0, '地下二層':0, '地下一層':0, '地下層':0, '全':100, '一層':1, '二層':2, '三層':3, '四層':4, '五層':5, '六層':6, '七層':7, '八層':8, '九層':9, '十層':10, '十一層':11, '十二層':12, '十三層':13, '十四層':14, '十五層':15, '十六層':16, '十七層':17, '十八層':18, '十九層':19, '二十層':20, '二十一層':21, '二十二層':22, '二十三層':23, '二十四層':24, '二十五層':25, '二十六層':26, '二十七層':27, '二十八層':28, '二十九層':29, '三十層':30, '三十一層':31, '三十二層':32, '三十三層':33, '三十四層':34, '三十五層':35, '三十六層':36, '三十七層':37, '三十八層':38, '三十九層':39, '四十層':40, '四十一層':41, '四十二層':42, '四十三層':43, '四十四層':44, '四十五層':45, '四十六層':46, '四十七層':47, '四十八層':48, '四十九層':49, '五十層':50}

In [32]:
f = f.replace({"總樓層數": column_dict})

In [33]:
def change(n1):
  if pd.isna(n1):
    return n1
  else:
    x = n1.split("，")[0].strip()
    if x.find(",") != -1:
      return x.split(",")[0].strip()
    else:
      return x

In [34]:
f["移轉層次"] = f["移轉層次"].apply(change)

In [35]:
f = f.replace({"移轉層次": column_dict})

# 處理主要用途

In [36]:
# 填補主要用途缺失值

f["主要用途"] = f["主要用途"].fillna("其他用途")

In [39]:
# 將主要用途的其他合併到其他用途

s = f["主要用途"]
for i in range(len(f)):
  if s[i] == "其他":
    s[i] = s[i].replace("其他", "其他用途")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# 處理車位類別

In [42]:
f["車位類別"].isna().sum()

1127

In [43]:
f["車位類別"] = f["車位類別"].fillna("無車位")

In [44]:
f["車位類別"].value_counts()

無車位     1127
坡道平面     547
坡道機械      80
塔式車位      50
升降機械      44
升降平面      39
其他        32
一樓平面       2
Name: 車位類別, dtype: int64

In [45]:
# 將車位類別的其他改成其他車位類別

s = f["車位類別"]
for i in range(len(f)):
  if s[i] == "其他":
    s[i] = s[i].replace("其他", "其他車位類別")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# 處理主要建材

In [47]:
f["主要建材"].value_counts()

鋼筋混凝土造      1670
鋼骨造           94
鋼骨鋼筋混凝土造      76
加強磚造          50
鋼骨混凝土造        10
預力混凝土造         5
磚造             5
見使用執照          5
土磚石混合造         2
混凝土造           1
︰鐵筋水泥磚造        1
鋼骨ＲＣ造          1
木造             1
Name: 主要建材, dtype: int64

In [48]:
# 填補主要建材缺失值

most = f["主要建材"].value_counts().idxmax()
f["主要建材"] = f["主要建材"].fillna(most)

In [49]:
# 將主要用途的其他合併到其他用途

s = f["主要建材"]
for i in range(len(f)):
  if s[i] != "鋼筋混凝土造":
    f = f.drop(i)

# 重設index
f = f.reset_index(drop=True)

# 處理TF欄位(有無管理組織、電梯)

In [ ]:
tf_dict = {"有":1, "無":0}

# 將有無管理組織轉成0和1
f = f.replace({"有無管理組織": tf_dict})

# 將電梯轉成0和1
f = f.replace({"電梯": tf_dict})

# 處理交易年月日、建築完成年月

In [55]:
# 將交易年份取出，新增到交易年欄位

f["交易年"] = 0
for i in range(len(f)):
  f.at[i, "交易年"] = f["交易年月日"][i][0:3]

In [56]:
# 如果有的話，將建築完成年份取出，新增到建築年欄位

f["建築年"] = 0
for i in range(len(f)):
  if f["建築完成年月"].isna()[i] == False:
   f.at[i, "建築年"] = f["建築完成年月"][i][0:3]

In [57]:
# 將建築年=0的值帶入建築年的中位數

for i in range(len(f)):
  if f["建築年"][i] == 0:
    f.at[i, "建築年"] = f["建築年"].median()

In [58]:
# 新增屋齡欄位，並將交易年-建築年，得到屋齡

f["屋齡"] = 0
for i in range(len(f)):
  f.at[i, "屋齡"] = int(f["交易年"][i]) - int(f["建築年"][i])

In [59]:
# 去除屋齡<0的房屋

for i in range(len(f)):
  if f["屋齡"][i] <= 0:
    f = f.drop(i)

f = f.reset_index(drop=True)

# 新增年平均CPI、CPI年增率欄位

In [77]:
from google.colab import files
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Tibame_03期_第二組團專/整理後的資料') #切換該目錄
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['公費快篩診所.csv',
 '博物館.csv',
 '醫療 Final.csv',
 '教育 Final.csv',
 '消防 Final.csv',
 'shilin_111_house.csv',
 '圖書館 Final.csv',
 '金融 Final.csv',
 '郵局 Final.csv',
 '機場 Final.csv',
 '禮儀 Final.csv',
 '廢棄物處理場 Final.csv',
 '所得.csv',
 '台北市房屋交易_1108_含通膨.csv',
 '捷運出口 Final.csv',
 '警察Final.csv',
 '公園Final.csv',
 '教育程度.csv',
 '消費者物價指數銜接表.csv']

In [72]:
# 讀取消費者物價指數銜接表.csv檔案
df = pd.read_csv("消費者物價指數銜接表.csv", encoding = "utf-8")

In [73]:
# 製作每年CPI字典

cpi_dict = {}

for i in range(len(df)):
  cpi_dict[df["年"][i]] = df["累計平均"][i]

In [74]:
# 新增年平均CPI欄位，將對應CPI加入f表格中

f["年平均CPI"] = 0.0

for i in range (0, len(f)):
  if int(f["交易年"][i]) in cpi_dict:
    f.at[i, "年平均CPI"] = float(cpi_dict[int(f["交易年"][i])])

In [78]:
# 讀取消費者物價指數銜接表-年增率.csv檔案

df2 = pd.read_csv("消費者物價指數銜接表-年增率.csv", encoding = "utf-8")

In [79]:
# 製作每年CPI年增率字典

cpi_dict_inc = {}

for i in range(len(df2)):
  cpi_dict_inc[df2["年"][i]] = df2["全年"][i]

# 將111年的值從nan改成0

cpi_dict_inc[111] =0

In [80]:
# 新增CPI年增率欄位，將對應CPI年增率加入f表格中

f["CPI年增率"] = 0.0

for i in range (0, len(f)):
  if int(f["交易年"][i]) in cpi_dict_inc:
    f.at[i, "CPI年增率"] = float(cpi_dict_inc[int(f["交易年"][i])])

# 新增各區所得中位數欄位

In [81]:
# 讀取所得.csv檔案

df3 = pd.read_csv("所得.csv", encoding = "utf-8")

In [83]:
# 留下合計欄位

for i in range(len(df3)):
  if df3["村里"][i] != "合計":
    df3 = df3.drop(i)

# 重設index
df3 = df3.reset_index(drop=True)

In [84]:
# 製作所得中位數字典

med_dict = {}

for i in range(len(df3)):
  med_dict[df3["鄉鎮市區"][i]] = int(df3["中位數"][i]) * 1000

In [85]:
# 新增所得中位數欄位，將對應所得中位數加入f表格中

f["所得中位數"] = 0

for i in range(len(f)):
  if f["鄉鎮市區"][i] in med_dict:
    f.at[i, "所得中位數"] = med_dict[f["鄉鎮市區"][i]]

# 新增經度緯度欄位

In [86]:
pip install googlemaps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for googlemaps: filename=googlemaps-4.7.3-py3-none-any.whl size=40341 sha256=121294c782fc1dedb07757c5cf09a29c40f71a1d1445a54fe91ba43a7f777a36
  Stored in directory: /root/.cache/pip/wheels/e3/83/52/3aa3b54c0dd386c0f7e20c7af2748822d2cde581c7555939bd
Successfully built googlemaps


In [87]:
# 地址轉經緯度
import googlemaps

In [88]:
f["緯度"] = 0.0
f["經度"] = 0.0

for i in range(0, len(f)):  
  gmaps = googlemaps.Client(key="") # 輸入google maps API 金鑰
  address = f["土地位置建物門牌"][i]
  geocode = gmaps.geocode(address=address)
  (lat, lng) = map(geocode[0]['geometry']['location'].get, ('lat', 'lng'))

  f["緯度"][i] = (lat, lng)[0]
  f["經度"][i] = (lat, lng)[1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


# 儲存檔案

In [101]:
fname = input("請輸入文件年份季度，EX: 111S1 ")
f.to_csv("/content/drive/My Drive/Tibame_03期_第二組團專/整理後的資料/每年每季房屋交易資料/台北市房屋交易_"+str(fname)+".csv", encoding="utf-8", index=False)

請輸入文件年份季度，EX: 111S1 111S1
